In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


In [4]:
dat = pd.read_csv("data/train.csv")
X_orig = dat.loc[:, dat.columns != "target"]
y_orig = dat.loc[:, "target"]

In [5]:
X_no_neg = X_orig
y_no_neg = y_orig
cat = pd.DataFrame()
bn = pd.DataFrame()
norm = pd.DataFrame()
ids = X_orig['id']

## We ignore the id column for this step
for i in X_orig.columns[1:]:
    if "cat" in i:
        # Transform Median
        cat[i] = dat.loc[:, i]
    elif "bin" in i:
        # Transform
        bn[i] = dat.loc[:, i]
    else:
        # Do something
        norm[i] = dat.loc[:, i]
        
for i in cat.columns:
    tmp = 0
    cat.loc[cat[i] == -1, i] = np.median(cat.loc[cat[i] != -1,i])
    tmp = tmp + sum(cat[i] == -1)
    
#print("Number of -1 in data:", tmp)

for i in bn.columns:
    tmp = 0
    tmp = tmp + sum(bn[i] == -1)
    print(i, ":", tmp)
    
#print("There are no negatives in bin columns")
temp = []
for feature in cat:
    temp.append(pd.get_dummies(cat[feature]))

new_cat = pd.DataFrame()
for feature in temp:
    new_cat = pd.concat([new_cat, feature], axis=1)

mx = 0
for i in norm.columns:
    tmp = 0
    tmp = tmp + sum(norm[i] == -1)
    print(i, ":", tmp)
    if mx < tmp:
        mx=tmp

print("~", mx/float(len(norm)) * 100, "% of the data is missing, so we are going to assume that there is significance that the data is missing")
num_missing_per_row = np.zeros(len(norm))
tmp = 0
for index, row in norm.iterrows():
    num_missing_per_row[index] =  sum(row == -1)
    
pipe = Pipeline([
       ("remove_neg_ones", Imputer(missing_values=-1, strategy="mean")),
        ("z-scaling", StandardScaler())
        ])

scaled_norm = pd.DataFrame(pipe.fit_transform(norm), columns = norm.columns)
X_final = pd.concat([bn, new_cat, scaled_norm], axis=1)
X_orig = X_final

ps_ind_06_bin : 0
ps_ind_07_bin : 0
ps_ind_08_bin : 0
ps_ind_09_bin : 0
ps_ind_10_bin : 0
ps_ind_11_bin : 0
ps_ind_12_bin : 0
ps_ind_13_bin : 0
ps_ind_16_bin : 0
ps_ind_17_bin : 0
ps_ind_18_bin : 0
ps_calc_15_bin : 0
ps_calc_16_bin : 0
ps_calc_17_bin : 0
ps_calc_18_bin : 0
ps_calc_19_bin : 0
ps_calc_20_bin : 0
ps_ind_01 : 0
ps_ind_03 : 0
ps_ind_14 : 0
ps_ind_15 : 0
ps_reg_01 : 0
ps_reg_02 : 0
ps_reg_03 : 107772
ps_car_11 : 5
ps_car_12 : 1
ps_car_13 : 0
ps_car_14 : 42620
ps_car_15 : 0
ps_calc_01 : 0
ps_calc_02 : 0
ps_calc_03 : 0
ps_calc_04 : 0
ps_calc_05 : 0
ps_calc_06 : 0
ps_calc_07 : 0
ps_calc_08 : 0
ps_calc_09 : 0
ps_calc_10 : 0
ps_calc_11 : 0
ps_calc_12 : 0
ps_calc_13 : 0
ps_calc_14 : 0
~ 18.1064897885 % of the data is missing, so we are going to assume that there is significance that the data is missing


In [72]:
X_train, X_test, y_train, y_test = train_test_split(X_orig, y_orig)

In [39]:
print("Type:", type(X_train), "Len", len(X_train), "Column len", len(X_train.columns), "orig column len", 
      len(X_orig.columns))
print("Type:", type(y_train), "Len", len(y_train))

Type: <class 'pandas.core.frame.DataFrame'> Len 446409 Column len 218 orig column len 218
Type: <class 'pandas.core.series.Series'> Len 446409


In [53]:
#boolList = np.asarray(y_orig==1)

X_ones = X_train.loc[y_train==1, :]
#idx = np.random.randint(len(X_ones) * 2, size=2)
X_ones = pd.DataFrame(X_ones)
X_ones = X_ones.sample(n=(len(X_ones)*4), replace=True)
y_ones = y_train[y_train==1]
X_zeroes = X_train.loc[y_train==0, :]
X_zeroes = pd.DataFrame(X_zeroes)
dat_ones = X_ones
dat_ones["target"] = [1] * (len(X_ones))
dat_f = pd.DataFrame(X_zeroes.sample(n=(len(X_ones)), replace=True))
dat_f["target"] = np.zeros(len(dat_f))
dat_ones = dat_ones.append(dat_f.sample(n=(len(dat_ones))))

In [41]:
#X_ones = X_orig.loc[y_orig, :]
#y_ones = y_orig[y_orig==1]
#X_zeroes = X_orig.loc[y_orig==0, :]
#dat_ones = pd.DataFrame(X_ones)
#dat_ones["target"] = [1] * len(dat_ones)
#dat_f = pd.DataFrame(X_zeroes)
#dat_f["target"] = np.zeros(len(dat_f))
#dat_ones = dat_ones.append(dat_f.sample(n=(len(dat_ones)), replace=True))
#X_shrunk = dat_ones.loc[:, dat_ones.columns != "target"]
#y_shrunk = dat_ones.target



#dat_ones.target.unique()

array([ 1.,  0.])

In [54]:
X_shrunk = dat_ones.loc[:, dat_ones.columns != "target"]
y_shrunk = dat_ones.target

In [55]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
     assert( len(actual) == len(pred) )
     all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
     all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
     totalLosses = all[:,0].sum()
     giniSum = all[:,0].cumsum().sum() / totalLosses
 
     giniSum -= (len(actual) + 1) / 2
     return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [73]:
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
predicted = clf.predict_proba(X_test)
#confusion_matrix(y_test, np.round(predicted))
#print(np.amin(predicted))

In [74]:
print(f"Gini score for default GradientBoostingRegressor {gini_normalized(y_test, predicted[:,1])}")

Gini score for default GradientBoostingRegressor 0.28252667475253185


Difference of splits are large (.004 in change)